In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#libs
import matplotlib.pyplot as plt
from matplotlib import colormaps
import numpy as np
import os
import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms.functional import pil_to_tensor
import gc

from torchmetrics import JaccardIndex

#our classes
import utils #contains sam_utils, visual_utils, and other utility functions
from datasets.dataset_loading import CocoLoader 

#sam
from segment_anything import SamPredictor, sam_model_registry, SamAutomaticMaskGenerator
from segment_anything.utils.transforms import ResizeLongestSide

/home.stud/svobo114/.conda/envs/detect_env_clone/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#CUDA tests
os.environ["CUDA_VISIBLE_DEVICES"] = "1" #might not be viable, check again!

print("CUDA available:" + str(torch.cuda.is_available()))
from torch.utils.cpp_extension import CUDA_HOME
print("CUDA_HOME:" + str(CUDA_HOME))
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
print(available_gpus)
print(torch.cuda.device_count())

CUDA available:True
CUDA_HOME:/home.stud/svobo114/.conda/envs/detect_env_clone
1


In [3]:
coco=CocoLoader() 
transforms = None
data_train, api = coco.load_train(transformations=transforms)

loading annotations into memory...
Done (t=18.38s)
creating index...
index created!
loading annotations into memory...
Done (t=9.29s)
creating index...
index created!


In [4]:
predictor,sam = utils.prepare_sam("cuda")
resize_transform = ResizeLongestSide(sam.image_encoder.img_size)

/home.stud/svobo114/.conda/envs/detect_env_clone/lib/python3.11/site-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = t

In [35]:
# Set batch size and number of workers
batch_size = 2
num_workers = 4
shuffle=True

# Clear CUDA cache
with torch.no_grad():
    torch.cuda.empty_cache()
gc.collect()

# Create DataLoader from the training dataset
data_loader =  DataLoader(
            data_train,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers,
            collate_fn=lambda x: tuple(zip(*x)),
        )

# 1 for 20 GB GPU with cca 7 boxes per image -> batch<=4, maybe 5 if lucky

In [53]:

#for a batch:
    #get boxes from the anns, + masks
    #load all images and prompts into a list for batch
    #sam() on the batch
    #take the best mask for each box
    #take scores etc
    #calculate IoU for each mask
    #recall?
    
#CUDA cleanup before running

torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
gc.collect()

dataset_IoU= JaccardIndex(task="binary")

#run batches
for i, batch in enumerate(data_loader):
    print("Batch: "+str(i))
    images_pil = list(batch[0])
    metadata = list(batch[1])

    #separate GT for metrics
    gt_boxes=[]
    gt_masks=[]
    for j in range(len(images_pil)):
        masks_img, boxes_img = utils.coco_masks_boxes(metadata[j], api) # passing api class from training dta, blank would suffice
        boxes_img = torch.Tensor(utils.boxes_coco_to_sam(boxes_img)) #change format and to tensor
        gt_boxes.append(boxes_img)
        gt_masks.append(masks_img)
    gt_masks=gt_masks
    gt_boxes=gt_boxes


    # TODO for now using GT boxes, later detection module HERE 
    inferrence_boxes=gt_boxes 

    #prepare input for batch
    sam_batched_inputs=[]
    for j in range(len(images_pil)): 
        img = np.array(images_pil[j])
        dict_img={
                    'image': utils.prepare_image_for_batch(img, resize_transform, sam.device),
                    'boxes': resize_transform.apply_boxes_torch(inferrence_boxes[j].to(sam.device),img.shape[:2]),
                    'original_size': img.shape[:2]
                    }
        sam_batched_inputs.append(dict_img)
    
    #run inference
    batched_output = sam(sam_batched_inputs, multimask_output=True) 

    #Take best masks in each image
    for j,dict_output in enumerate(batched_output):      #dict_keys(['masks', 'iou_predictions', 'low_res_logits'])
        pred_quality = dict_output['iou_predictions']
        best=np.argmax(pred_quality.cpu(),axis=1)
        
        arange=torch.arange(best.shape[0])
        best_masks = dict_output['masks'][arange,best]

        #calculate metrics
        dataset_IoU.update(best_masks.cpu(),torch.Tensor(gt_masks[j])) #both on cpu? 


    print("Max GB allocated: "+ str(torch.cuda.max_memory_allocated()//1000000000)+"."+str((torch.cuda.max_memory_allocated()%1000_000_000)//1_000_000))
    if i >=0: #just few batches for now
        print("Mean IoU: "+str(dataset_IoU.compute()))
        break


#CUDA cleanup after running
with torch.no_grad():
    torch.cuda.empty_cache()
gc.collect()
    

ValueError: `num_classes` is expected to be `int` but `<class 'NoneType'> was passed.`

In [33]:
print(1/0)

ZeroDivisionError: division by zero